In [2]:
!pip install mlflow
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import logging
import time
from datetime import datetime
import subprocess
import platform
import psutil
import os
import mlflow
import mlflow.pytorch
from sklearn.metrics import classification_report, confusion_matrix


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# Configure logging
format = '%(asctime)s - %(levelname)s - %(filename)s - PID:%(process)d - TID:%(thread)d - %(message)s'
logger = logging.getLogger(__name__ + str(time.time()))
logger.setLevel(logging.INFO)
logger.propagate = False

# Add handler if none exists
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter(format))
    logger.addHandler(handler)

In [4]:
# Data transforms for CIFAR-100
train_tf = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # Data augmentation
    transforms.RandomHorizontalFlip(0.5),   # Random horizontal flip
    transforms.ToTensor(),                  # Convert to tensor
    transforms.Normalize(                   # Normalize with ImageNet stats
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load CIFAR-100 dataset
train_ds = datasets.CIFAR100(
    root="./data",
    train=True,
    download=True,
    transform=train_tf
)

print(f"Training dataset size: {len(train_ds)}")
print(f"Number of classes: {len(train_ds.classes)}")
print(f"Class names (first 10): {train_ds.classes[:10]}")

Training dataset size: 50000
Number of classes: 100
Class names (first 10): ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle']


In [5]:
# Optional: Visualize a sample of the data
def visualize_samples(dataset, num_samples=16):
    """Visualize a sample of images from the dataset"""
    # Create subset for visualization
    sample_indices = np.random.choice(len(dataset), num_samples, replace=False)

    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    axes = axes.ravel()

    for i, idx in enumerate(sample_indices):
        img_tensor, class_idx = dataset[idx]

        # Denormalize image for display
        img = img_tensor.permute(1, 2, 0)
        img = img * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
        img = torch.clamp(img, 0, 1)

        axes[i].imshow(img)
        axes[i].set_title(f"Class: {dataset.classes[class_idx]}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Uncomment to visualize samples
# visualize_samples(train_ds)

In [6]:
# Create data loader
train_loader = DataLoader(
    train_ds,
    batch_size=128,
    shuffle=True,
    num_workers=0,      # Set to 0 for MPS compatibility
    pin_memory=False    # Disable for MPS
)

print(f"Number of batches per epoch: {len(train_loader)}")
print(f"Total samples per epoch: {len(train_loader) * 128}")

Number of batches per epoch: 391
Total samples per epoch: 50048


In [7]:
class ComprehensiveTrainingMonitor:
    """Enhanced training monitor with MLflow integration and comprehensive metrics"""

    def __init__(self, model, optimizer, criterion, device, model_name='ResNet18',
                 dataset_name='CIFAR-100', batch_size=128, epochs=100,
                 input_size=(3, 32, 32), use_mlflow=True,
                 learning_rate=5e-3, use_pretrained=True, train_size=50000,
                 val_size=10000, num_workers=0):

        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        self.model_name = model_name
        self.dataset_name = dataset_name
        self.batch_size = batch_size
        self.epochs = epochs
        self.input_size = input_size
        self.use_mlflow = use_mlflow
        self.learning_rate = learning_rate
        self.use_pretrained = use_pretrained
        self.train_size = train_size
        self.val_size = val_size
        self.num_workers = num_workers

        # Tracking variables
        self.best_metric = float('-inf')
        self.epoch_times = []
        self.start_time = time.time()
        self.run_started = False

        # MLflow configuration
        self.mlflow_uri = "https://neuralripper.com/mlflow/"

        if self.use_mlflow:
            self._initialize_mlflow()

    def _initialize_mlflow(self):
        """Initialize MLflow with comprehensive experiment tracking
        MLflow routes registry operations through your remote server instead of trying to write locally. 
        The server handles the separation:
        registry metadata goes to your database backend, artifacts go to S3.
        """
        try:
            mlflow.set_tracking_uri(self.mlflow_uri)
            mlflow.set_registry_uri(self.mlflow_uri) 
            mlflow.set_experiment(f"{self.model_name}-{self.dataset_name}")

            # Start run with timestamp
            run_name = f"{self.model_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            mlflow.start_run(run_name=run_name)
            self.run_started = True

            # Log all parameters
            params = {
                **self._get_model_params(),
                **self._get_system_params(),
                **self._get_environment_params(),
                **self._get_data_params(),
                **self._get_training_params(),
            }

            mlflow.log_params(params)
            logger.info(f"MLflow run started: {run_name}")

        except Exception as e:
            logger.warning(f"Failed to initialize MLflow: {e}")
            self.use_mlflow = False

    def _get_model_params(self):
        """Model architecture and hyperparameters"""
        total_params = sum(p.numel() for p in self.model.parameters())
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)

        return {
            "model_name": self.model_name,
            "total_parameters": total_params,
            "trainable_parameters": trainable_params,
            "model_type": "classification",
            "pretrained": self.use_pretrained,
            "batch_size": self.batch_size,
            "epochs": self.epochs,
            "learning_rate": self.learning_rate,
            "optimizer": self.optimizer.__class__.__name__,
            "criterion": self.criterion.__class__.__name__,
            "input_channels": self.input_size[0],
            "input_height": self.input_size[1],
            "input_width": self.input_size[2],
        }

    def _get_system_params(self):
        """System hardware and software parameters"""
        gpu_info = {}
        if torch.cuda.is_available():
            props = torch.cuda.get_device_properties(0)
            gpu_info = {
                "gpu_name": torch.cuda.get_device_name(0),
                "gpu_memory_gb": round(props.total_memory / (1024**3), 2),
                "cuda_version": torch.version.cuda,
                "num_gpus": torch.cuda.device_count(),
            }
        elif torch.backends.mps.is_available():
            gpu_info = {
                "gpu_name": "Apple Silicon MPS",
                "device_type": "mps"
            }

        return {
            "cpu_count": psutil.cpu_count(),
            "memory_total_gb": round(psutil.virtual_memory().total / (1024**3), 2),
            "platform": platform.platform(),
            "python_version": platform.python_version(),
            "pytorch_version": torch.__version__,
            **gpu_info
        }

    def _get_environment_params(self):
        """Environment and reproducibility parameters"""
        git_info = {}
        try:
            commit = subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode().strip()
            branch = subprocess.check_output(['git', 'rev-parse', '--abbrev-ref', 'HEAD']).decode().strip()
            git_info = {
                "git_commit": commit[:8],
                "git_branch": branch,
            }
        except:
            git_info = {"git_commit": "unknown", "git_branch": "unknown"}

        return git_info

    def _get_data_params(self):
        """Data pipeline parameters"""
        return {
            "train_size": self.train_size,
            "val_size": self.val_size,
            "total_samples": self.train_size + self.val_size,
            "num_workers": self.num_workers,
        }

    def _get_training_params(self):
        """Advanced training configuration"""
        return {
            "mlflow_uri": self.mlflow_uri,
            "experiment_name": f"{self.model_name}-{self.dataset_name}",
        }

    def log_epoch_metrics(self, epoch, epoch_loss, epoch_acc, batch_count=None):
        """Comprehensive epoch metrics logging"""
        if not self.use_mlflow or not self.run_started:
            return

        try:
            epoch_time = self.epoch_times[-1] if self.epoch_times else 0

            # Core metrics
            metrics = {
                "train_loss": epoch_loss,
                "train_accuracy": epoch_acc,
                "epoch_time_seconds": epoch_time,
                "learning_rate": self.optimizer.param_groups[0]['lr'],
            }

            # Performance metrics
            if batch_count:
                metrics["batches_per_second"] = batch_count / epoch_time if epoch_time > 0 else 0
                metrics["samples_per_second"] = (batch_count * self.batch_size) / epoch_time if epoch_time > 0 else 0

            # Memory metrics
            if torch.cuda.is_available():
                metrics["gpu_memory_allocated_gb"] = torch.cuda.memory_allocated() / (1024**3)
                metrics["gpu_memory_reserved_gb"] = torch.cuda.memory_reserved() / (1024**3)

            # Running statistics
            total_time = sum(self.epoch_times)
            metrics["total_time_minutes"] = total_time / 60
            metrics["average_epoch_time"] = np.mean(self.epoch_times) if self.epoch_times else 0

            mlflow.log_metrics(metrics, step=epoch)

            # Update best model if improved
            if epoch_acc > self.best_metric:
                self.best_metric = epoch_acc
                self._log_model_checkpoint(epoch)

        except Exception as e:
            logger.warning(f"Failed to log metrics for epoch {epoch}: {e}")

    def _log_model_checkpoint(self, epoch):
        """Log model checkpoint and metadata"""
        try:
            # Create sample input as numpy array (CPU)
            sample_input = torch.randn(1, *self.input_size)
            sample_input_np = sample_input.numpy()
            
            # Set model to eval mode before inference
            was_training = self.model.training
            self.model.eval()
            
            # Get model prediction and convert to numpy
            with torch.no_grad():
                sample_output = self.model(sample_input.to(self.device))
                sample_output_np = sample_output.cpu().numpy()
            
            # Restore training mode if it was in training
            if was_training:
                self.model.train()
            
            # Manually infer signature using numpy arrays
            from mlflow.models import infer_signature
            signature = infer_signature(sample_input_np, sample_output_np)
            
            # Log model with signature
            mlflow.pytorch.log_model(
                self.model,
                artifact_path="model",
                registered_model_name=f"{self.model_name}-{self.dataset_name}",
                signature=signature,
                pip_requirements=["torch", "torchvision", "pillow", "numpy"]
            )

            # Log as METRICS (not parameters) - these can be updated
            model_metrics = {
                "best_epoch_checkpoint": epoch,
                "best_accuracy_checkpoint": self.best_metric,
            }
            mlflow.log_metrics(model_metrics, step=epoch)
            
            # Log checkpoint timestamp as tag (tags can be updated)
            mlflow.set_tag("last_checkpoint_time", datetime.now().isoformat())
            
            logger.info(f"Model checkpoint logged for accuracy: {self.best_metric:.4f}")

        except Exception as e:
            logger.warning(f"Failed to log model checkpoint: {e}")

    def end_run(self, status="FINISHED"):
        """Clean up and end MLflow run"""
        if self.use_mlflow and self.run_started:
            try:
                total_time = time.time() - self.start_time
                summary = {
                    "final_total_training_time_minutes": round(total_time / 60, 2),
                    "final_best_accuracy": self.best_metric,
                    "final_epochs_completed": len(self.epoch_times),
                }
                mlflow.log_params(summary)
                mlflow.end_run(status=status)
                self.run_started = False
                return summary
            except Exception as e:
                logger.warning(f"Failed to end MLflow run properly: {e}")
                return {}
        return {}

In [8]:
class EnhancedResNet18:
    """ResNet18 with comprehensive monitoring integration"""

    def __init__(self, num_epochs=100, batch_size=128, num_classes=100,
                 learning_rate=5e-3, use_mlflow=True):
        # Core parameters
        self._num_classes = num_classes
        self._use_mlflow = use_mlflow
        self._batch_size = batch_size
        self._num_epochs = num_epochs
        self._learning_rate = learning_rate

        # Create model components
        self._model = self._create_model()
        self._device = self._set_device()
        self._model.to(self._device)
        self._criterion = self._set_criterion()
        self._optimizer = self._set_optimizer()

        # Initialize comprehensive monitor
        self.monitor = ComprehensiveTrainingMonitor(
            model=self._model,
            optimizer=self._optimizer,
            criterion=self._criterion,
            device=self._device,
            model_name='ResNet18',
            dataset_name='CIFAR-100',
            batch_size=batch_size,
            epochs=num_epochs,
            input_size=(3, 32, 32),
            learning_rate=learning_rate,
            use_mlflow=use_mlflow,
            use_pretrained=True,
            train_size=50000,
            val_size=10000,
            num_workers=0,
        )

        logger.info(f"Model initialized on device: {self._device}")
        logger.info(f"Model parameters: {sum(p.numel() for p in self._model.parameters()):,}")

    def _create_model(self):
        """Create ResNet18 model with CIFAR-100 adaptation"""
        model = resnet18(weights=ResNet18_Weights.DEFAULT)
        # Adapt classifier for CIFAR-100 (100 classes)
        model.fc = nn.Linear(model.fc.in_features, self._num_classes)
        return model

    def _set_device(self):
        """Set appropriate device for training"""
        if torch.backends.mps.is_available():
            return torch.device("mps")
        elif torch.cuda.is_available():
            return torch.device("cuda")
        return torch.device("cpu")

    def _set_optimizer(self):
        """Configure SGD optimizer with momentum and weight decay"""
        return optim.SGD(self._model.parameters(),
                        lr=self._learning_rate,
                        momentum=0.9,
                        weight_decay=4e-5)

    def _set_criterion(self):
        """Set loss function for classification"""
        return nn.CrossEntropyLoss()

    def train_epoch(self, data_loader, epoch_idx):
        """Enhanced training epoch with comprehensive monitoring"""
        logger.info(f"Starting epoch {epoch_idx+1}, total batches: {len(data_loader)}")

        self._model.train()
        epoch_total_loss = 0.0
        running_loss = 0.0
        running_correct = 0
        running_total = 0
        batch_count = len(data_loader)

        for idx, (images, targets) in enumerate(data_loader):
            images = images.to(self._device)
            targets = targets.to(self._device)

            self._optimizer.zero_grad()
            logits = self._model(images)
            loss = self._criterion(logits, targets)
            loss.backward()
            self._optimizer.step()

            predictions = logits.argmax(dim=1)
            running_correct += (predictions == targets).sum().item()
            running_total += targets.size(0)
            running_loss += loss.item()
            epoch_total_loss += loss.item()

            # Log progress every 10 batches
            if idx % 10 == 9:
                avg_loss = running_loss / 10
                acc_sofar = running_correct / running_total
                logger.info(f"Epoch {epoch_idx+1} | Batch {idx+1}/{batch_count} | "
                          f"Loss {avg_loss:.4f} | Acc {acc_sofar:.4f}")
                running_loss = 0.0

        # Calculate epoch metrics
        epoch_loss = epoch_total_loss / batch_count
        epoch_acc = running_correct / running_total

        logger.info(f"Epoch {epoch_idx+1} completed | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")
        return epoch_loss, epoch_acc

    def train(self, data_loader):
        """Full training loop with comprehensive monitoring"""
        logger.info("Starting training process")

        try:
            for epoch in range(self._num_epochs):
                epoch_start = time.time()

                # Train for one epoch
                epoch_loss, epoch_acc = self.train_epoch(data_loader, epoch)

                # Track timing
                epoch_time = time.time() - epoch_start
                self.monitor.epoch_times.append(epoch_time)

                # Log metrics
                self.monitor.log_epoch_metrics(epoch, epoch_loss, epoch_acc, len(data_loader))

                # Progress report
                eta = np.mean(self.monitor.epoch_times) * (self._num_epochs - epoch - 1)
                logger.info(f"Epoch {epoch+1}/{self._num_epochs} | "
                          f"Time: {epoch_time:.1f}s | ETA: {eta/60:.1f}min")

        except KeyboardInterrupt:
            logger.info("Training interrupted by user")
            summary = self.monitor.end_run(status="KILLED")
            return summary
        except Exception as e:
            logger.error(f"Training failed with error: {e}")
            summary = self.monitor.end_run(status="FAILED")
            raise e

        # Training completed
        summary = self.monitor.end_run(status="FINISHED")
        logger.info(f"Training completed. Summary: {summary}")

        if summary:
            print(f"\nTraining Summary:")
            print(f"Total time: {summary.get('final_total_training_time_minutes', 0):.1f} minutes")
            print(f"Best accuracy: {summary.get('final_best_accuracy', 0):.4f}")
            print(f"Epochs completed: {summary.get('final_epochs_completed', 0)}")

        return summary

In [11]:
# Clear any existing MLflow runs
import mlflow

# End any active runs
try: 
    if mlflow.active_run():
        print(f"Ending active run: {mlflow.active_run().info.run_id}")
        mlflow.end_run()
        print("Active run ended")
    else:
        print("No active runs to clear")
except:
    mlflow.end_run(status="KILLED")

# Verify no active runs
print(f"Active run after cleanup: {mlflow.active_run()}")

# Initialize and train the model
logger.info("Initializing Enhanced ResNet18 model")

model = EnhancedResNet18(
    num_epochs=200,
    batch_size=256,
    num_classes=100,
    learning_rate=10e-3,
    use_mlflow=True
)

print("Model initialized successfully!")
print(f"Device: {model._device}")
print(f"Total parameters: {sum(p.numel() for p in model._model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model._model.parameters() if p.requires_grad):,}")

2025-08-14 23:17:05,248 - INFO - 2951892819.py - PID:14177 - TID:8467341504 - Initializing Enhanced ResNet18 model


No active runs to clear
Active run after cleanup: None


2025-08-14 23:17:05,560 - WARNING - 3467894994.py - PID:14177 - TID:8467341504 - Failed to initialize MLflow: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 401 != 200. Response body: '<html>
<head><title>401 Authorization Required</title></head>
<body>
<center><h1>401 Authorization Required</h1></center>
<hr><center>nginx/1.29.0</center>
</body>
</html>
'
2025-08-14 23:17:05,561 - INFO - 2563978282.py - PID:14177 - TID:8467341504 - Model initialized on device: mps
2025-08-14 23:17:05,561 - INFO - 2563978282.py - PID:14177 - TID:8467341504 - Model parameters: 11,227,812


Model initialized successfully!
Device: mps
Total parameters: 11,227,812
Trainable parameters: 11,227,812


In [ ]:
# Start training with comprehensive monitoring
print("\n" + "="*50)
print(" STARTING ENHANCED RESNET18 TRAINING")
print("="*50)
print(f"Dataset: CIFAR-100 (100 classes)")
print(f"Model: ResNet18 (pretrained)")
print(f"Epochs: {model._num_epochs}")
print(f"Batch size: {model._batch_size}")
print(f"Learning rate: {model._learning_rate}")
print(f"MLflow tracking: {'Enabled' if model._use_mlflow else 'Disabled'}")
print(f"MLflow URI: {model.monitor.mlflow_uri}")
print("="*50 + "\n")

# Start training
model.train(train_loader)